In [ ]:
from fastapi import FastAPI, Response, Request
from starlette.responses import StreamingResponse
import cv2
import time
import uvicorn
import numpy as np
from PIL import ImageGrab
import datetime
import asyncio
from queue import Queue
import nest_asyncio
import io
from mss import mss
from PIL import Image, ImageDraw
import pyautogui
nest_asyncio.apply()
app = FastAPI()

# Shared variable to store the latest frame
latest_frame = None

async def generate_frames():
    global latest_frame
    # Variable for frame interval (in seconds) to achieve 1 FPS
    frame_interval = 1
    start_time = time.time()
    num_frames = 0
    while True:
        frame_start_time = time.time()
        img = ImageGrab.grab()  # Capture the screen
        img = np.array(img)
        img_final = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        if True:
            img_final = cv2.resize(img_final, (1920, 1080)) 
            _, buffer = cv2.imencode('.jpg', img_final)
            num_frames += 1

            latest_frame = buffer.tobytes()  # Update the latest frame

        # Calculate elapsed time
        elapsed_time = time.time() - start_time

        # If one second has elapsed, calculate and print FPS
        if elapsed_time >= 1:
            fps = num_frames / elapsed_time
            # print("FPS:", fps)
            num_frames = 0
            start_time = time.time()

        frame_time = time.time() - frame_start_time
        # Introduce the delay to achieve 1 FPS
        delay = frame_interval - frame_time
        try:
            await asyncio.sleep(delay)
        except asyncio.CancelledError:
            break


# Load and resize the mouse icon once
mouse_icon = Image.open("mouse_icon.png")
n = 26
new_size = (n, n)
mouse_icon = mouse_icon.resize(new_size, Image.NEAREST)
icon_width, icon_height = mouse_icon.size

# Create an mss instance
sct = mss()


def capture_screen_with_mouse():
    s = time.time()
    
    # Get the current mouse cursor position
    mouse_x, mouse_y = pyautogui.position()
    position = [mouse_x, mouse_y]
    print(f"Mouse position: ({mouse_x}, {mouse_y})")
    
    # Capture the screen
    monitor = {"top": 0, "left": 0, "width": 1920, "height": 1080}  # Adjust these values to match your screen resolution
    sct_img = sct.grab(monitor)
    screenshot = Image.frombytes("RGB", (sct_img.width, sct_img.height), sct_img.rgb)
    
    # Calculate the position to paste the mouse icon
    icon_position = (mouse_x - icon_width // 2, mouse_y - icon_height // 2)
    
    # Create a drawing context
    draw = ImageDraw.Draw(screenshot)
    
    # Draw the inner blue rectangle around the mouse icon
    box_padding = 2  # Adjust this value to change the thickness of the box
    inner_box_position = (
        icon_position[0] - box_padding,
        icon_position[1] - box_padding,
        icon_position[0] + icon_width + box_padding,
        icon_position[1] + icon_height + box_padding
    )
    # draw.rectangle(inner_box_position, outline="blue", width=box_padding)
    
    # Draw the outer rectangle 30 pixels away from the inner rectangle
    outer_padding = 30
    outer_box_position = (
        inner_box_position[0] - outer_padding,
        inner_box_position[1] - outer_padding,
        inner_box_position[2] + outer_padding,
        inner_box_position[3] + outer_padding
    )
    draw.rectangle(outer_box_position, outline="red", width=box_padding)
    
    # Paste the mouse icon onto the screenshot
    screenshot.paste(mouse_icon, icon_position, mouse_icon)
    
    e = time.time()
    r = e - s
    print(f'total {r}')
    
    return screenshot,position

# Global variables
sct = mss()
mouse_icon = Image.open("mouse_icon.png").resize((26, 26), Image.NEAREST)
icon_width, icon_height = mouse_icon.size
monitor = {"top": 0, "left": 0, "width": 1920, "height": 1080}

def capture_screenshot():
    # Capture the screen
    sct_img = sct.grab(monitor)
    img = Image.frombytes("RGB", (sct_img.width, sct_img.height), sct_img.rgb)
    
    # Get the current mouse position
    mouse_x, mouse_y = pyautogui.position()
    
    # Calculate mouse icon position
    icon_position = (mouse_x - icon_width // 2, mouse_y - icon_height // 2)
    
    # Paste the mouse icon onto the screenshot
    img.paste(mouse_icon, icon_position, mouse_icon)
    
    return img

@app.get("/screenshot")
async def get_screenshot():
    img = capture_screenshot()
    
    # Convert the image to JPEG format to reduce size
    img_byte_arr = io.BytesIO()
    img.save(img_byte_arr, format='JPEG', quality=70)  # Reduced quality for speed
    img_byte_arr.seek(0)
    
    # Return the image as a response
    return Response(content=img_byte_arr.getvalue(), media_type="image/jpeg")


@app.get("/video_feed")
async def video_feed():
    async def stream_frame():
        global latest_frame
        while True:
            start_time = time.time()  # Record start time of sending frames
            for _ in range(1):  # Send 1 frame per second
                if latest_frame is not None:  # Check if there's a latest frame available
                    yield (
                        b'--frame\r\n'
                        b'Content-Type: image/jpeg\r\n\r\n' + latest_frame + b'\r\n'
                    )
                else:
                    await asyncio.sleep(0.15)  # If no latest frame available, wait for a short time

            # Calculate elapsed time and wait if necessary to maintain 1 FPS rate
            elapsed_time = time.time() - start_time
            if elapsed_time < 1:
                await asyncio.sleep(1 - elapsed_time)

    return StreamingResponse(stream_frame(), media_type="multipart/x-mixed-replace; boundary=frame")




# async def main():
#     # Start generating frames asynchronously
#     asyncio.create_task(generate_frames())













from fastapi.responses import JSONResponse
import asyncio
import pyautogui

@app.post("/receive_stream")
async def receive_stream(request: Request):
    received_text = ""
    async for chunk in request.stream():
        received_text += chunk.decode()
        content = chunk.decode()
        pyautogui.typewrite(content)
        print(content, end='', flush=True)  # Print each chunk as it's received
    if len(received_text)!=0:
        pyautogui.press('enter')
    # return JSONResponse({"received": received_text})
    

@app.post('/send_position')
async def frames_meaning_endpoint(data: dict):
    global x_y_position
    x_y_position = data.get('dahwin')
    
    print(x_y_position)
    # Extract the x and y values from the text
    # Split the text at ', ' and then split each part at '=' to get the numbers
    x, y = x_y_position.split(', ')
    x = int(x.split('=')[1])
    y = int(y.split('=')[1])
    print(x, y)
    
    # Click at the extracted coordinates
    pyautogui.click(x, y)


if __name__ == "__main__":
    # asyncio.run(main())
    uvicorn.run(app, host="localhost", port=3000)

INFO:     Started server process [13028]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://localhost:3000 (Press CTRL+C to quit)


In [1]:
received_text = ""
len(received_text)

0

In [4]:
import requests

# Define the text to send
text = "I love Dahyun"

# Define the URL of the FastAPI server
url = "https://45c3-35-230-8-174.ngrok-free.app/receive-text/"

# Send a POST request to the server
response = requests.post(url, json={"text": text})

# Print the response from the server
print("Server response:", response.json())


Server response: None


In [ ]:
import asyncio
import io
import time
from typing import Generator

import cv2
import numpy as np
import pyautogui
import uvicorn
from fastapi import FastAPI, Request
from fastapi.responses import JSONResponse, StreamingResponse
from mss import mss
from PIL import Image, ImageDraw
import nest_asyncio
nest_asyncio.apply()
app = FastAPI()

# Global variables
sct = mss()
mouse_icon = Image.open("mouse_icon.png").resize((26, 26), Image.NEAREST)
icon_width, icon_height = mouse_icon.size
monitor = {"top": 0, "left": 0, "width": 1920, "height": 1080}

# FPS tracking variables
fps_count = 0
fps_start_time = time.time()

async def generate_frames() -> Generator[bytes, None, None]:
    global fps_count, fps_start_time
    total_fps = 0
    while True:
        total_fps+=1
        print(f"total fps {total_fps}")
        # Capture and process frame
        sct_img = sct.grab(monitor)
        img = Image.frombytes("RGB", (sct_img.width, sct_img.height), sct_img.rgb)

        # Add mouse cursor
        mouse_x, mouse_y = pyautogui.position()
        icon_position = (mouse_x - icon_width // 2, mouse_y - icon_height // 2)
        img.paste(mouse_icon, icon_position, mouse_icon)

        # Convert to numpy array and encode to JPEG
        img_array = np.array(img)
        img_bgr = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)
        
        # # Add FPS text to the image
        # cv2.putText(img_bgr, f"FPS: {fps_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
        _, buffer = cv2.imencode('.jpg', img_bgr, [cv2.IMWRITE_JPEG_QUALITY, 80])
        frame = buffer.tobytes()

        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

        # Update FPS count
        fps_count += 1
        current_time = time.time()
        elapsed_time = current_time - fps_start_time

        # If a second has passed, update FPS metrics
        if elapsed_time >= 1:
            print(f"FPS: {fps_count}")
            fps_count = 0
            fps_start_time = current_time

        await asyncio.sleep(0.001)  # Small sleep to prevent CPU overuse

@app.get("/video_feed")
async def video_feed():
    return StreamingResponse(generate_frames(), media_type="multipart/x-mixed-replace; boundary=frame")

@app.get("/screenshot")
async def get_screenshot():
    sct_img = sct.grab(monitor)
    img = Image.frombytes("RGB", (sct_img.width, sct_img.height), sct_img.rgb)

    # Add mouse cursor
    mouse_x, mouse_y = pyautogui.position()
    icon_position = (mouse_x - icon_width // 2, mouse_y - icon_height // 2)
    img.paste(mouse_icon, icon_position, mouse_icon)

    # Draw red rectangle around cursor
    draw = ImageDraw.Draw(img)
    rect_size = 60
    rect_position = (
        mouse_x - rect_size // 2,
        mouse_y - rect_size // 2,
        mouse_x + rect_size // 2,
        mouse_y + rect_size // 2
    )
    draw.rectangle(rect_position, outline="red", width=2)

    # Convert to bytes
    img_byte_arr = io.BytesIO()
    img.save(img_byte_arr, format='PNG')
    img_byte_arr = img_byte_arr.getvalue()

    headers = {
        "Content-Disposition": "attachment; filename=screenshot.png",
        "Mouse-Position-X": str(mouse_x),
        "Mouse-Position-Y": str(mouse_y)
    }

    return StreamingResponse(io.BytesIO(img_byte_arr), media_type="image/png", headers=headers)

@app.post("/receive_stream")
async def receive_stream(request: Request):
    received_text = ""
    async for chunk in request.stream():
        received_text += chunk.decode()
        content = chunk.decode()
        pyautogui.typewrite(content)
        print(content, end='', flush=True)  # Print each chunk as it's received
    
    # return JSONResponse({"received": received_text})
    pyautogui.press('enter')

@app.post('/send_position')
async def frames_meaning_endpoint(data: dict):
    global x_y_position
    x_y_position = data.get('dahwin')
    
    print(x_y_position)
    # Extract the x and y values from the text
    # Split the text at ', ' and then split each part at '=' to get the numbers
    x, y = x_y_position.split(', ')
    x = int(x.split('=')[1])
    y = int(y.split('=')[1])
    print(x, y)
    
    # Click at the extracted coordinates
    pyautogui.click(x, y)


if __name__ == "__main__":
    uvicorn.run(app, host="localhost", port=3000)

INFO:     Started server process [2716]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://localhost:3000 (Press CTRL+C to quit)


x=1256, y=500
1256 500
INFO:     34.172.242.125:0 - "POST /send_position HTTP/1.1" 200 OK
x=1256, y=500
1256 500
INFO:     34.172.242.125:0 - "POST /send_position HTTP/1.1" 200 OK


In [ ]:
from fastapi import FastAPI, Response, Request
from starlette.responses import StreamingResponse
import cv2
import time
import uvicorn
import numpy as np
from PIL import ImageGrab
import datetime
import asyncio
from queue import Queue
import io
from fastapi.responses import JSONResponse
import asyncio

from mss import mss
from PIL import Image, ImageDraw
import pyautogui
import nest_asyncio
nest_asyncio.apply()

app = FastAPI()

@app.get("/")
async def serve_text():
    return {"text":   "https://0a26-34-91-73-91.ngrok-free.app"  }


if __name__ == "__main__":
    uvicorn.run(app, host="localhost", port=9999)

INFO:     Started server process [17436]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://localhost:9999 (Press CTRL+C to quit)


INFO:     27.147.202.138:0 - "GET / HTTP/1.1" 200 OK


In [ ]:
sudo cloudflared service uninstall

In [ ]:
sudo cloudflared service install eyJhIjoiNmU0MzkzMDc0ZjJmODRjMDkwYzUwNDNjZTc1NmEyMTciLCJ0IjoiZDAwMWRiNWEtYjc5OS00NTFlLThlNzUtMjM0ZGE4Mjc5ZDhkIiwicyI6Ik5HSXpOelZtTXprdE9EbGpaUzAwWVROa0xXSXhOalF0TURKaVlqYzNaRFZrTW1NdyJ9

In [3]:
import requests

def fetch_text():
    url ="https://agi.queendahyun.site"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad status codes
        data = response.json()
        print("Received text:", data["text"])
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")

if __name__ == "__main__":
    fetch_text()

Received text: https://9264-35-193-132-99.ngrok-free.app
